<a href="https://colab.research.google.com/github/mohamedh-2000/MindolifeTest/blob/master/photoshop_Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# התקנת הספריות הדרושות
!pip install rembg
!pip install gradio
!pip install numpy
!pip install opencv-python-headless
!pip install pillow
!pip install onnxruntime

# ייבוא הספריות
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from rembg import remove
from io import BytesIO
import gradio as gr

# פונקציה לזיהוי פנים
def detect_faces(image):
    # המרת התמונה ל-Numpy Array
    image_np = np.array(image.convert("RGB"))
    # המרת התמונה ל-BGR עבור OpenCV
    image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
    # שימוש ב-Haar Cascade לזיהוי פנים
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(image_bgr, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
    # החזרת True אם זוהו פנים, אחרת False
    return len(faces) > 0

# פונקציה להסרת רקע
def remove_background(image):
    image_data = BytesIO()
    image.save(image_data, format="PNG")
    result = remove(image_data.getvalue())
    return Image.open(BytesIO(result))

# פונקציה לשילוב דמות עם רקע
def combine_person_with_background(person_image, background_image, size_factor=0.4):
    try:
        # בדיקת זיהוי פנים בתמונה הראשונה
        if not detect_faces(person_image):
            return (
                None,
                '<div style="text-align: center; color: red; font-size: 20px; font-weight: bold; '
                'background-color: yellow; padding: 20px; border-radius: 10px;">'
                'Error: No face detected in the person image. Please upload a valid image containing a face.'
                '</div>'
            )

        # בדיקת זיהוי פנים בתמונה השנייה
        if not detect_faces(background_image):
            return (
                None,
                '<div style="text-align: center; color: red; font-size: 20px; font-weight: bold; '
                'background-color: yellow; padding: 20px; border-radius: 10px;">'
                'Error: No face detected in the background image. Please upload a valid image with a clear background.'
                '</div>'
            )

        # הסרת רקע מהדמות
        person_no_bg = remove_background(person_image).convert("RGBA")

        # קבלת מידות הדמות והרקע
        person_width, person_height = person_no_bg.size
        bg_width, bg_height = background_image.size

        # התאמת גודל הדמות לפי פקטור מותאם
        new_width = int(bg_width * size_factor)
        new_height = int(new_width * (person_height / person_width))
        person_no_bg = person_no_bg.resize((new_width, new_height), Image.LANCZOS)

        # מציאת מיקום מתאים במרכז הרקע
        position = (bg_width // 2 - new_width // 2, bg_height // 2 - new_height // 2)

        # הדבקת הדמות על הרקע
        composite = background_image.copy()
        composite.paste(person_no_bg, position, person_no_bg)

        # שיפור צבעים ותאורה
        enhancer = ImageEnhance.Color(composite)
        final_image = enhancer.enhance(1.2)

        return final_image, ""
    except Exception as e:
        return None, f'<div style="text-align: center; color: red; font-size: 20px;">Error: {e}</div>'

# יצירת ממשק Gradio
interface = gr.Interface(
    fn=combine_person_with_background,
    inputs=[
        gr.Image(type="pil", label="Upload Person Image"),
        gr.Image(type="pil", label="Upload Background Image"),
        gr.Slider(0.1, 1.0, step=0.1, value=0.4, label="Size Factor (Proportion of Background)")
    ],
    outputs=[
        gr.Image(type="pil", label="Blended Image"),
        gr.HTML(label="Message")
    ],
    title="Smart Image Combiner",
    description="Upload a person image and a background image. The tool will verify the first image contains a face and combine it with the background."
)

# הפעלת הממשק
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e6f44dfbd9809f5284.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
